In [75]:
from __future__ import print_function
import torch as t
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
        
    def forward(self,x):
        
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        
        x = x.view(x.size()[0],-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

In [76]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [77]:
params = list(net.parameters())
print(len(params))

10


In [78]:
for name,parameters in net.named_parameters():
    print(name,':',parameters.size())

conv1.weight : torch.Size([6, 1, 5, 5])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 5, 5])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 400])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([84, 120])
fc2.bias : torch.Size([84])
fc3.weight : torch.Size([10, 84])
fc3.bias : torch.Size([10])


In [79]:
input = Variable(t.randn(1,1,32,32))
out = net(input)
out.size

<function Tensor.size>

In [80]:
net.zero_grad()
out.backward(Variable(t.ones(1,10)))

In [81]:
output = net(input)
target = Variable(t.arange(0,10))
target = target.float()

In [83]:
criterion = nn.MSELoss()
loss = criterion(output,target)
loss

tensor(28.6743, grad_fn=<MseLossBackward>)

In [84]:
net.zero_grad()
print('Grad before')
print(net.conv1.bias.grad)
loss.backward()
print('Grad after')
print(net.conv1.bias.grad)

Grad before
tensor([0., 0., 0., 0., 0., 0.])
Grad after
tensor([-0.1029, -0.0004,  0.0591,  0.0006, -0.0588, -0.0279])


In [89]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(),lr = 0.01)

optimizer.zero_grad()

output = net(input)
loss = criterion(output,target)

loss.backward()

optimizer.step()